In [ ]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import kospi_predict as kp

In [ ]:
# df_kodex = pd.read_clipboard()
# df_kodex = df_kodex[:-2]

# df_kodex.rename(columns={"날짜" : "date", "종가" : "KODEX_CLS", "오픈" : "KODEX_OPN"}, inplace=True)
# df_kodex["KODEX_OPN"] = df_kodex["KODEX_OPN"].str.replace(",", "")
# df_kodex["KODEX_CLS"] = df_kodex["KODEX_CLS"].str.replace(",", "")
# df_kodex["KODEX_OPN"] = df_kodex["KODEX_OPN"].astype(int)
# df_kodex["KODEX_CLS"] = df_kodex["KODEX_CLS"].astype(int)

# df_kodex["KODEX_GAP"] = df_kodex["KODEX_OPN"] - df_kodex["KODEX_CLS"]
# df_kodex.sort_values("date", inplace=True)
# df_kodex.reset_index(drop=True, inplace=True)

# for item in df_kodex.iterrows():
#     new_ymd = item[1]["date"].replace("년 ", "-").replace("월 ", "-").replace("일", "")
#     month = new_ymd.split("-")[1]
#     day = new_ymd.split("-")[2]
#     if len(month) == 1:
#         month = "0"+month
#     if len(day) == 1:
#         day = "0"+day
#     new_ymd = "{}-{}-{}".format(new_ymd.split("-")[0], month, day)
#     df_kodex.loc[item[0], "date"] = new_ymd
# df_kodex.to_excel("crawled_data_kodex.xlsx", index=False)
# df_kodex

In [ ]:
# df_kodex_inv = pd.read_clipboard()
# df_kodex_inv = df_kodex_inv[:-2]

# df_kodex_inv.rename(columns={"날짜" : "date", "종가" : "KODEX_INV_CLS", "오픈" : "KODEX_INV_OPN"}, inplace=True)
# df_kodex_inv["KODEX_INV_OPN"] = df_kodex_inv["KODEX_INV_OPN"].str.replace(",", "")
# df_kodex_inv["KODEX_INV_CLS"] = df_kodex_inv["KODEX_INV_CLS"].str.replace(",", "")
# df_kodex_inv["KODEX_INV_OPN"] = df_kodex_inv["KODEX_INV_OPN"].astype(int)
# df_kodex_inv["KODEX_INV_CLS"] = df_kodex_inv["KODEX_INV_CLS"].astype(int)

# df_kodex_inv["KODEX_INV_GAP"] = df_kodex_inv["KODEX_INV_OPN"] - df_kodex_inv["KODEX_INV_CLS"]
# df_kodex_inv.sort_values("date", inplace=True)
# df_kodex_inv.reset_index(drop=True, inplace=True)

# for item in df_kodex_inv.iterrows():
#     new_ymd = item[1]["date"].replace("년 ", "-").replace("월 ", "-").replace("일", "")
#     month = new_ymd.split("-")[1]
#     day = new_ymd.split("-")[2]
#     if len(month) == 1:
#         month = "0"+month
#     if len(day) == 1:
#         day = "0"+day
#     new_ymd = "{}-{}-{}".format(new_ymd.split("-")[0], month, day)
#     df_kodex_inv.loc[item[0], "date"] = new_ymd
# df_kodex_inv.to_excel("crawled_data_kodex_inv.xlsx", index=False)
# df_kodex_inv

In [ ]:
df_kodex = pd.read_excel("crawled_data_kodex.xlsx")
df_kodex = df_kodex.loc[:, ["date", "KODEX_CLS", "KODEX_OPN", "KODEX_GAP"]]
df_kodex

In [ ]:
df_kodex_inv = pd.read_excel("crawled_data_kodex_inv.xlsx")
df_kodex_inv = df_kodex_inv.loc[:, ["date", "KODEX_INV_CLS", "KODEX_INV_OPN", "KODEX_INV_GAP"]]
df_kodex_inv

In [ ]:
df_kospi = pd.read_excel("crawled_data.xlsx")
df_kospi.dropna(inplace=True)
df_kospi.sort_values("date", inplace=True)
df_kospi.reset_index(drop=True, inplace=True)
df_kospi = df_kospi.iloc[:, 1:6]
df_kospi

In [ ]:
df = df_kospi.merge(df_kodex, how="right", on="date")
df = df.merge(df_kodex_inv, how="left", on="date")
df.dropna(inplace=True)
df.sort_values("date", inplace=True)

# df = df.iloc[100:600]

df.reset_index(drop=True, inplace=True)
df

In [ ]:
state_list = []
state_list_df = pd.DataFrame()
for idx in range(len(df)):
    if idx > 59:
        this_mat = df.iloc[idx-60:idx, 0:5].to_numpy()
        for col_idx in range(1, this_mat.shape[1]):
            this_arr = this_mat[:, col_idx]
            max_val = max(this_arr)
            min_val = min(this_arr)
            for row_idx, val in enumerate(this_arr):
                new_val = (val - min_val) / (max_val - min_val)
                this_mat[row_idx, col_idx] = new_val
        state_list.append(this_mat)

state_list = np.array(state_list)
state_list.shape

In [ ]:
action_names = ["KD매수", "KDI매수"]

In [ ]:
model_pred = tf.keras.Sequential([
    tf.keras.layers.Input((state_list.shape[1], state_list.shape[2]-1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation="relu"),
    tf.keras.layers.Dense(units=128, activation="relu"),
    tf.keras.layers.Dense(units=128, activation="relu"),
    tf.keras.layers.Dense(units=128, activation="relu"),
    tf.keras.layers.Dense(units=128, activation="relu"),
    tf.keras.layers.Dense(units=2, activation="softmax")
])

In [ ]:
model_pred.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss="mse", 
              metrics=["mae", "mse"])

model_pred.summary()
model_pred.save("model_pred.h5")
model_q = tf.keras.models.load_model("model_pred.h5")

In [ ]:
len(state_list)

In [ ]:
update_period = 32 #스테이지 32회당 1번씩 pred 모델의 웨이트를 업데이트 한다.
copy_period = 64 #스테이지 64회당 1번씩 q모델을 pred 모델로부터 복제시킨다.
episodes = 300
result = []
for episode in range(1, episodes+1):
    
    #변수 초기화
    capital = 1000 * 10000 #천만원으로 시작
    remain_cash = capital
    buy_cost_kodex = 0
    buy_cost_kodex_inv = 0
    batch_buffer = []
    reward_list = []
    update_cnt = 0
    copy_cnt = 0

    for idx, state in enumerate(state_list):
        update_cnt += 1
        copy_cnt += 1

        #스테이지 횟수가 10회가 되면 pred 모델을 학습시키고, 학습시킨 모델을 q 모델에 복제한다.
        if update_cnt+1 == update_period:
            batch_buffer = np.array(batch_buffer, dtype=np.float64)
            reward_list = np.array(reward_list, dtype=np.float64)

            model_pred.fit(batch_buffer, reward_list, verbose=0, callbacks = tf.keras.callbacks.TensorBoard(log_dir="log"))
            model_pred.save("model_pred.h5")
            batch_buffer = [] #업데이트 한 후 미니배치(x)를 초기화한다.
            reward_list = [] #업데이트 한 후 보상값(y)을 초기화한다.
            update_cnt = 0
            # print("모델 업데이트")

        if copy_cnt+1 == copy_period:
            model_q = tf.keras.models.load_model("model_pred.h5") #q모델을 pred모델로부터 복제해 온다.
            copy_cnt = 0
            # print("모델 복제")

        state_data = np.array(state[:, 1:], dtype=np.float64)
        batch_buffer.append(state_data) #pred 학습을 위해 먼저 미니배치 버퍼에 x를 넣어놓는다.

        state_data = np.expand_dims(state_data, 0) #추론을 위해 축 추가

        #Q 모델에게 물어본다.
        pred = model_q.predict(state_data)
        action_selected = np.argmax(pred[0])
        action_name_org = action_names[action_selected]

        #exploration을 위해 초기에는 랜덤하게 액션을 선택해 본다.
        # rand_action_no = random.randrange(0, 2)
        # if episode < episodes*0.5:
        #     rand_action_select = random.randrange(0, 2)
        #     if idx < len(state_list)*0.8 and rand_action_select == 1:
        #         action_selected = rand_action_no
        # elif episode < episodes*0.7:
        #     rand_action_select = random.randrange(0, 4)
        #     if idx < len(state_list)*0.5 and rand_action_select == 1:
        #         action_selected = rand_action_no
        # elif episode < episodes*0.8:
        #     rand_action_select = random.randrange(0, 10)
        #     if idx < len(state_list)*0.3 and rand_action_select == 1:
        #         action_selected = rand_action_no

        action_name = action_names[action_selected]

        #오늘의 데이터와 다음날의 데이터를 불러온다.
        today_data = df.loc[(df["date"] == state[-1, 0])]
        tomorrow_idx = df.loc[(df["date"] == state[-1, 0])].index[0] + 1
        tomorrow_data = df.loc[tomorrow_idx]

        #액션을 취한다 (다음날 시작 가격으로 매수 혹은 매도)
        if action_name == "KD매수":
            buy_cnt = np.floor(remain_cash / tomorrow_data["KODEX_OPN"])
            buy_cost_kodex = buy_cnt * tomorrow_data["KODEX_OPN"]
            reward = (buy_cnt * (tomorrow_data["KODEX_CLS"] - tomorrow_data["KODEX_OPN"]))

        elif action_name == "KDI매수":
            buy_cnt = np.floor(remain_cash / tomorrow_data["KODEX_INV_OPN"])
            buy_cost_kodex_inv = buy_cnt * tomorrow_data["KODEX_INV_OPN"]
            reward = (buy_cnt * (tomorrow_data["KODEX_INV_CLS"] - tomorrow_data["KODEX_INV_OPN"]))


        remain_cash = remain_cash + reward
        
        #다음날의 보상을 reward_list에 반영한다.
        buy_cnt_kdx_expected = np.floor(capital / tomorrow_data["KODEX_OPN"])
        buy_cnt_inv_expected = np.floor(capital / tomorrow_data["KODEX_INV_OPN"])

        action1_reward_expected = (buy_cnt_kdx_expected * (tomorrow_data["KODEX_CLS"] - tomorrow_data["KODEX_OPN"]))
        action2_reward_expected = (buy_cnt_inv_expected * (tomorrow_data["KODEX_INV_CLS"] - tomorrow_data["KODEX_INV_OPN"]))

        this_reward_list = [
            action1_reward_expected,
            action2_reward_expected
        ]
        #각 액션별 예상 보상값이 이게 맞는지......(나중에 해결하길!!!!!!!!!!!)
        #Q(s, a) = (1-alpha)(r) + alpha(gamma(q(s+1, a+1))) --> 이걸로 넣어줘야............(나중에....)

        this_reward_list[action_selected] = reward
        reward_list.append(this_reward_list)

        print("\n\n[EPISODE{}, S{} : {} → {}] {} {}".format(
            episode, idx, action_name_org, action_name, pred[0], this_reward_list), end="\r")
        print("\n잔고 : {:,d}원".format(int(remain_cash), end="\r"))

        #잔고가 100만원 이하가 되면 terminal 상태로 간주, 에피소드를 종료한다.
        if remain_cash < (100 * 10000):
            print("\r에피소드 종료(state {}) : {}, {}원".format(idx, action_name, remain_cash))
            result.append({
                "EPISODE" : episode,
                "잔고" : remain_cash
            })
            print(result)
            break

df_result = pd.DataFrame(result)
df_result